In [1]:
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor, BaggingRegressor, RandomForestRegressor, HistGradientBoostingRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.svm import SVR, NuSVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import RANSACRegressor, LinearRegression, HuberRegressor, SGDRegressor, GammaRegressor, PoissonRegressor, PassiveAggressiveRegressor, TheilSenRegressor, BayesianRidge, ARDRegression, TweedieRegressor, PassiveAggressiveClassifier, ElasticNet
from sklearn.neural_network import MLPRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.cross_decomposition import PLSRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from tqdm import tqdm
from sklearn.model_selection import cross_validate

In [2]:
all = pd.read_csv('../data/raw/all.csv')
all = all.drop(columns=['IMR', 'ID'])

all['MRG'] = all['MRG'].apply(lambda x: x if x in [91, 92, 93, 94, 95, 96, 97] else 90)

for k in all['REL'].value_counts().keys()[-1:]:
    all = all[all['REL'] != k]

for k in all['WORKPLACE'].value_counts().keys()[-2:]:
    all = all[all['WORKPLACE'] != k]

ITM40_mean, ITM40_std = all['ITM40'].mean(), all['ITM40'].std()
all['ITM40'] = (all['ITM40'] - ITM40_mean) / ITM40_std
all

,YEAR,PERSON,REL,SEX,AGE,EDU,IND,OCC,WKCLASS,WORK,WORKPLACE,MRG,PT,ITM40
0,2011,1,1,1,52,4,94,7,3,1,2,90,2,0.347617
1,2011,1,1,1,17,6,55,5,2,1,2,91,2,-0.626388
2,2011,2,1,2,35,4,55,5,2,1,2,90,2,-0.639083
3,2011,1,1,1,72,1,3,63,3,1,2,90,2,-0.622557
4,2011,1,1,2,47,3,8,8,2,1,2,94,2,-0.687500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290884,2021,3,3,2,24,7,64,3,2,1,63,91,2,-0.230407
290885,2021,51,3,2,21,5,55,5,2,1,17,91,2,-0.654031
290886,2021,1,5,2,61,6,0,0,8,2,0,97,2,-0.794130
290887,2021,31,1,2,36,8,45,3,2,1,63,91,2,-0.348243


In [3]:
X, Y = all.iloc[:, :-1], all.iloc[:, -1] 

In [4]:
df_encoded = pd.get_dummies(X, columns=list(X.columns), prefix='Prefix')
df_encoded

,Prefix_2011,Prefix_2012,Prefix_2013,Prefix_2014,Prefix_2015,Prefix_2016,Prefix_2017,Prefix_2018,Prefix_2019,Prefix_2020,...,Prefix_90,Prefix_91,Prefix_92,Prefix_93,Prefix_94,Prefix_95,Prefix_96,Prefix_97,Prefix_1,Prefix_2
0,True,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
1,True,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
2,True,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
3,True,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290884,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
290885,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
290886,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True
290887,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True


In [5]:
Y

0         0.347617
1        -0.626388
2        -0.639083
3        -0.622557
4        -0.687500
            ...   
290884   -0.230407
290885   -0.654031
290886   -0.794130
290887   -0.348243
290888   -0.062631
Name: ITM40, Length: 290870, dtype: float64

In [6]:
model_ExtraTreesRegressor = ExtraTreesRegressor()
model_RandomForestRegressor = RandomForestRegressor()
model_BaggingRegressor = BaggingRegressor()
model_HistGradientBoostingRegressor = HistGradientBoostingRegressor()
model_GradientBoostingRegressor = GradientBoostingRegressor()
model_DecisionTreeRegressor = DecisionTreeRegressor()
model_SVR = SVR()
model_NuSVR = NuSVR()
model_ExtraTreeRegressor = ExtraTreeRegressor()
model_MLPRegressor = MLPRegressor()
model_KNeighborsRegressor = KNeighborsRegressor()
model_LinearSVR = LinearSVR()
model_HuberRegressor = HuberRegressor()
model_TheilSenRegressor = TheilSenRegressor()
model_BayesianRidge = BayesianRidge()
model_KernelRidge = KernelRidge()
model_LinearRegression = LinearRegression()
model_SGDRegressor = SGDRegressor()
model_ARDRegression = ARDRegression()
model_PoissonRegressor = PoissonRegressor()
model_GammaRegressor = GammaRegressor()
model_RANSACRegressor = RANSACRegressor()
model_PLSRegression = PLSRegression()
model_PassiveAggressiveRegressor = PassiveAggressiveRegressor()
model_TweedieRegressor = TweedieRegressor()
model_AdaBoostRegressor = AdaBoostRegressor()
model_PassiveAggressiveClassifier = PassiveAggressiveClassifier()
model_ElasticNet = ElasticNet()
model_GaussianProcessRegressor = GaussianProcessRegressor()



model_list = [model_ExtraTreesRegressor,model_RandomForestRegressor,model_BaggingRegressor,model_HistGradientBoostingRegressor,model_GradientBoostingRegressor,model_DecisionTreeRegressor,model_SVR,model_NuSVR,model_ExtraTreeRegressor,model_MLPRegressor,model_KNeighborsRegressor,model_LinearSVR,model_HuberRegressor,model_TheilSenRegressor,model_BayesianRidge,model_KernelRidge,model_LinearRegression,model_SGDRegressor,model_ARDRegression,model_PoissonRegressor,model_GammaRegressor,model_RANSACRegressor,model_PLSRegression,model_PassiveAggressiveRegressor,model_TweedieRegressor,model_AdaBoostRegressor,model_PassiveAggressiveClassifier,model_ElasticNet,model_GaussianProcessRegressor]

In [7]:
result = []
for model in tqdm(model_list):
    result.append([model.__class__.__name__, 1, 2])
    continue
    res = cross_validate(model, X, Y, n_jobs=-1, verbose=1, return_train_score=True, scoring='neg_mean_absolute_error')
    test_score = -res['test_score'].mean()
    train_score = -res['train_score'].mean()
    result.append([model.__class__.__name__, test_score, train_score])

100%|██████████| 29/29 [00:00<?, ?it/s]


In [8]:
pd.DataFrame(result, columns=["name", "test_score", "train_score"]).to_csv("models")